Задание 3 Реализация классической архитектуры сверточных сетей LeNet-5

1) Подключение библиотек

In [11]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader

2) Настройка вычислений

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

torch.manual_seed(42)

device: cuda


3) Загрузка подготовленного датасета

In [13]:
DATA_PATH = r"..\data\prepared\notmnist_32.npz"
assert os.path.exists(DATA_PATH), f"File not found: {DATA_PATH}"

d = np.load(DATA_PATH)

X_train = torch.tensor(d["X_train"], dtype=torch.float32)
y_train = torch.tensor(d["y_train"], dtype=torch.long)

X_val = torch.tensor(d["X_val"], dtype=torch.float32)
y_val = torch.tensor(d["y_val"], dtype=torch.long)

X_test = torch.tensor(d["X_test"], dtype=torch.float32)
y_test = torch.tensor(d["y_test"], dtype=torch.long)

print("train:", X_train.shape, y_train.shape)
print("val:  ", X_val.shape, y_val.shape)
print("test: ", X_test.shape, y_test.shape)


train: torch.Size([415752, 1, 32, 32]) torch.Size([415752])
val:   torch.Size([46194, 1, 32, 32]) torch.Size([46194])
test:  torch.Size([13649, 1, 32, 32]) torch.Size([13649])


4) Формирование DataLoader

In [14]:
BATCH_SIZE = 128

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True,  num_workers=0)
val_loader   = DataLoader(TensorDataset(X_val,   y_val),   batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader  = DataLoader(TensorDataset(X_test,  y_test),  batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

xb, yb = next(iter(train_loader))
print("batch:", xb.shape, yb.shape)

batch: torch.Size([128, 1, 32, 32]) torch.Size([128])


5) Архитектура сети LeNet-5

In [15]:
class LeNet5(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0)    

        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
       
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0)
        
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)
        
        self.act = nn.Tanh()

    def forward(self, x):
        x = self.act(self.conv1(x))
        x = self.pool1(x)

        x = self.act(self.conv2(x))
        x = self.pool2(x)

        x = self.act(self.conv3(x))

        x = torch.flatten(x, 1)
        x = self.act(self.fc1(x))
        x = self.fc2(x)
        return x

6) Проверка корректности размерностей

In [16]:
model = LeNet5().to(device)

xb, yb = next(iter(train_loader))
xb = xb.to(device)

out = model(xb)
print("out shape:", out.shape)

out shape: torch.Size([128, 10])


7) Оптимизатор и гиперпараметры обучения

In [17]:
LR = 1e-3
EPOCHS = 30

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

8) Функции обучения и оценки качества

In [18]:
def accuracy(logits, y):
    preds = logits.argmax(dim=1)
    return (preds == y).float().mean().item()


@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    total_loss, total_acc, n = 0.0, 0.0, 0

    for X, y in loader:
        X, y = X.to(device), y.to(device)
        logits = model(X)
        loss = F.cross_entropy(logits, y)

        bs = X.size(0)
        total_loss += loss.item() * bs
        total_acc += accuracy(logits, y) * bs
        n += bs

    return total_loss / n, total_acc / n


def train_one_epoch(model, loader, optimizer):
    model.train()
    total_loss, total_acc, n = 0.0, 0.0, 0

    for X, y in loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(X)
        loss = F.cross_entropy(logits, y)
        loss.backward()
        optimizer.step()

        bs = X.size(0)
        total_loss += loss.item() * bs
        total_acc += accuracy(logits, y) * bs
        n += bs

    return total_loss / n, total_acc / n

9) Обучение модели LeNet-5

In [19]:
model = LeNet5().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer)
    val_loss, val_acc = evaluate(model, val_loader)

    print(f"Epoch {epoch:02d}: "
          f"train loss={train_loss:.4f}, acc={train_acc:.4f} | "
          f"val loss={val_loss:.4f}, acc={val_acc:.4f}")


Epoch 01: train loss=0.4831, acc=0.8551 | val loss=0.3813, acc=0.8837
Epoch 02: train loss=0.3481, acc=0.8928 | val loss=0.3338, acc=0.8974
Epoch 03: train loss=0.3153, acc=0.9028 | val loss=0.3199, acc=0.9012
Epoch 04: train loss=0.2964, acc=0.9079 | val loss=0.3049, acc=0.9054
Epoch 05: train loss=0.2834, acc=0.9117 | val loss=0.3015, acc=0.9073
Epoch 06: train loss=0.2731, acc=0.9142 | val loss=0.2964, acc=0.9076
Epoch 07: train loss=0.2645, acc=0.9172 | val loss=0.2971, acc=0.9077
Epoch 08: train loss=0.2576, acc=0.9188 | val loss=0.2939, acc=0.9107
Epoch 09: train loss=0.2510, acc=0.9211 | val loss=0.2910, acc=0.9110
Epoch 10: train loss=0.2454, acc=0.9224 | val loss=0.2870, acc=0.9124
Epoch 11: train loss=0.2403, acc=0.9240 | val loss=0.2891, acc=0.9122
Epoch 12: train loss=0.2360, acc=0.9253 | val loss=0.2895, acc=0.9121
Epoch 13: train loss=0.2318, acc=0.9266 | val loss=0.2889, acc=0.9123
Epoch 14: train loss=0.2276, acc=0.9278 | val loss=0.2910, acc=0.9130
Epoch 15: train loss

10) Оценка качества на тестовой выборке

In [20]:
test_loss, test_acc = evaluate(model, test_loader)
print(f"TEST: loss={test_loss:.4f}, acc={test_acc:.4f}")

TEST: loss=0.1518, acc=0.9575


В третьем задании была реализована классическая архитектура LeNet-5 для входных изображений 32×32. В ходе обучения наблюдается устойчивый рост качества: точность на обучающей выборке увеличилась с 0.855 до 0.940. Точность на валидационной выборке достигла уровня около 0.914 и стабилизировалась примерно после 10 эпох, что свидетельствует о достижении плато. Значительное расхождение между обучающей и валидационной точностью отсутствует, следовательно, выраженного переобучения не наблюдается. По результатам тестирования получена точность 0.9575, что является лучшим результатом среди трёх рассмотренных моделей, подтверждая эффективность классической архитектуры LeNet-5 при решении задачи классификации изображений notMNIST.